In [ ]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import webbrowser
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Load Data
vehicles_df = pd.read_csv('vehicles_df.csv')

# Load Reviews Data (Kia Motors specific)
reviews_df = pd.read_csv('car_reviews_with_sentiment_and_percentage.csv')

# Initialize Dash App
app = dash.Dash(__name__)

# Geocode cities to get latitude and longitude
geolocator = Nominatim(user_agent="geoapi")
city_coordinates = []

for city in city_counts['City'].unique():
    try:
        location = geolocator.geocode(f"{city}, Illinois")  # Update state/region as needed
        if location:
            city_coordinates.append({
                "City": city,
                "latitude": location.latitude,
                "longitude": location.longitude
            })
        else:
            city_coordinates.append({
                "City": city,
                "latitude": None,
                "longitude": None
            })
    except GeocoderTimedOut:
        city_coordinates.append({
            "City": city,
            "latitude": None,
            "longitude": None
        })

app.layout = html.Div([
    # Title and Introduction
    html.Div([
        html.H1("Vehicle Hierarchy Dashboard", style={'textAlign': 'center'}),
        html.P(
            "This dashboard provides an interactive view of vehicle registrations, categorized by their "
            "make, model, and color. Use the dropdown below to filter by fuel type and explore the hierarchy.",
            style={'textAlign': 'center', 'fontSize': '18px', 'marginBottom': '20px'}
        ),
        html.P(
            "Hover over the segments in the sunburst chart to view detailed information about the vehicles.",
            style={'textAlign': 'center', 'fontSize': '16px', 'color': 'white'}
        )
    ], style={'padding': '20px', 'backgroundColor': '#769199'}),
        # Dropdown Filter
    html.Div([
        html.Label("Filter by Fuel Type:", style={'fontSize': '16px', 'fontWeight': 'bold', 'color': 'black', 'marginBottom': '10px'}),
        dcc.Dropdown(
            id='fuel-filter',
            options=[{'label': ft, 'value': ft} for ft in vehicles_df['Vehicle Fuel Source'].dropna().unique()],
            value=vehicles_df['Vehicle Fuel Source'].dropna().unique()[0],
            clearable=False
        )
    ], style={'padding': '20px'}),

    # Sunburst Chart
    html.Div([
        dcc.Graph(id='sunburst-chart'),
        html.P(
            "This chart represents the hierarchy of vehicles based on their make, model, and color. "
            "The size of each segment indicates the number of vehicles within that category.",
            style={'textAlign': 'center', 'fontSize': '14px', 'marginTop': '10px'}
        )
    ]),
    # City Map
    html.Div([
        html.H3("City Distribution of Vehicle Registrations"),
        dcc.Graph(id='city-map'),
        html.P(
            "This map shows the distribution of vehicle registrations across cities. The size of the markers represents the number of registrations.",
            style={'textAlign': 'center', 'fontSize': '14px', 'marginTop': '10px'}
        )
    ]),
    #vehicle count br chart
    html.Div([
    html.H3("Vehicle Count by Make"),
    dcc.Graph(id='bar-chart'),
]),

# Dropdown Filter
    html.Div([
        html.Label("Filter by Sentiment:", style={'fontSize': '16px', 'fontWeight': 'bold', 'color': 'black'}),
        dcc.Dropdown(
            id='sentiment-dropdown',
            options=[
                {'label': 'All Sentiments', 'value': 'all'},
                {'label': 'Positive', 'value': 'POSITIVE'},
                {'label': 'Negative', 'value': 'NEGATIVE'},
                {'label': 'Neutral', 'value': 'neutral'}
            ],
            value='all',  # Default value
            clearable=False
        )
    ], style={'padding':'20px'}),
              
# Pie Chart for Review Classification
    html.Div([
        html.H3("Review Topics"),
        dcc.Graph(id='classification-pie-chart'),
    ], style={'padding':'20px'}),
# Bar Chart for Sentiment Analysis
    html.Div([
        html.H3("Sentiment Analysis"),
        dcc.Graph(id='sentiment-bar-chart'),
    ], style={'padding':'20px'}),
                           
    # Footer
    html.Div([
        html.P(
            "This dashboard is for demonstration purposes only and is not connected to a live database. "
            "The data used here is a sample dataset and does not reflect actual vehicle registrations.",
            style={'textAlign': 'center', 'fontSize': '14px', 'color': 'white'}
        )
    ], style={'padding': '20px', 'backgroundColor': '#769199'})
])
@app.callback(
    [Output('sunburst-chart', 'figure'),
    ],
    [Output('city-map', 'figure')],
    Output('bar-chart', 'figure'),
    Output('classification-pie-chart', 'figure'),
    [Input('fuel-filter', 'value')]
)

def update_charts(fuel_type):
    # Filter DataFrame based on the selected fuel type
    filtered_data = vehicles_df[vehicles_df['Vehicle Fuel Source'] == fuel_type].copy()
    
    # Handle missing values explicitly with .loc
    filtered_data.loc[:, 'Vehicle Make'] = filtered_data['Vehicle Make'].fillna('Unknown')
    filtered_data.loc[:, 'Vehicle Model'] = filtered_data['Vehicle Model'].fillna('Unknown')
    filtered_data.loc[:, 'Vehicle Color'] = filtered_data['Vehicle Color'].fillna('Unknown')
    
    # Add a Count column for the hierarchy visualization
    filtered_data.loc[:, 'Count'] = 1
    
    # Filter data based on the selected sentiment
    if selected_sentiment != 'all':
        filtered_df = reviews_df[reviews_df['sentiment'] == selected_sentiment]
    else:
        filtered_df = reviews_df

    # Pie Chart: Review Classification
    classification_data = filtered_df['talks_about'].value_counts().reset_index()
    classification_data.columns = ['Category', 'Count']
    pie_chart = px.pie(
        classification_data,
        names='Category',
        values='Count',
        title="Topics Discussed in Reviews"
)

    # Sunburst Chart
    sunburst = px.sunburst(
        filtered_data,
        path=['Vehicle Make', 'Vehicle Model', 'Vehicle Color'],
        values='Count',
        title="Vehicle Hierarchy by Make, Model, and Color",
        hover_data={
            'Vehicle Make': True,
            'Vehicle Model': True,
            'Vehicle Color': True,
            'Count': True
        }
    )
    sunburst.update_layout(
        margin=dict(t=50, l=25, r=25, b=25),
        hoverlabel=dict(
            bgcolor="black",
            font_size=14,
            font_family="Arial"
        )
    )
    # City Map
    city_map = px.scatter_mapbox(
        city_data,
        lat="latitude",
        lon="longitude",
        size="Count",
        color="Count",
        hover_name="City",
        title="City Distribution of Vehicle Registrations",
        mapbox_style="carto-positron",
        zoom=10
    )

    # Bar Chart: Vehicle Count by Make
vehicle_make_data = filtered_data.groupby('Vehicle Make').size().reset_index(name='Count')
bar_chart = px.bar(
    vehicle_make_data,
    x='Vehicle Make',
    y='Count',
    title="Vehicle Count by Make",
    text='Count',
    labels={'Count': 'Number of Vehicles'},
    color='Count',
    color_continuous_scale='Blues'
)
bar_chart.update_layout(
    xaxis_title="Vehicle Make",
    yaxis_title="Count",
    margin=dict(t=50, l=25, r=25, b=25),
    xaxis_tickangle=-45
)

    # Bar Chart: Vehicle Count by Make
vehicle_make_data = filtered_data.groupby('Vehicle Make').size().reset_index(name='Count')
bar_chart = px.bar(
        vehicle_make_data,
        x='Vehicle Make',
        y='Count',
        title="Vehicle Count by Make",
        text='Count',
        labels={'Count': 'Number of Vehicles'},
        color='Count',
        color_continuous_scale='Blues'
    )
bar_chart.update_layout(
        xaxis_title="Vehicle Make",
        yaxis_title="Count",
        margin=dict(t=50, l=25, r=25, b=25),
        xaxis_tickangle=-45
    )

    # Bar Chart: Sentiment Analysis
sentiment_data = filtered_df.groupby('sentiment').agg(
        {'Review': 'count', 'sentiment_percentage': 'mean'}
    ).reset_index()
sentiment_data.columns = ['Sentiment', 'Count', 'Average Sentiment Percentage']
sentiment_bar_chart = px.bar(
        sentiment_data,
        x='Sentiment',
        y='Count',
        text='Average Sentiment Percentage',
        title="Customer Sentiment Distribution",
        color='Sentiment'
    )
sentiment_bar_chart.update_traces(texttemplate='%{text:.2f}%', textposition='outside') 

# Run the App
if __name__ == '__main__':
        webbrowser.open_new('http://127.0.0.1:3000/')
        app.run_server(debug=True, host='127.0.0.1', port=3000)

FileNotFoundError: [Errno 2] No such file or directory: 'car_reviews_with_sentiment_and_percentage.csv'

In [ ]:
html.Div([
            dcc.Graph(id='histogram-chart'),
            html.P(
                "This histogram illustrates the distribution of vehicle model years, helping to identify the "
                "most common manufacturing years among registered vehicles.",
                style={'textAlign': 'center', 'color': '#7E909A'}
            )
        ], style={'width': '48%', 'display': 'inline-block'}),

histogram = px.histogram(
        filtered_data, 
        x='Vehicle Model Year', 
        nbins=20, 
        title="Vehicle Model Year Distribution", 
        labels={'Vehicle Model Year': 'Year'}, 
        color_discrete_sequence=['#A5D8DD']
    )

return sunburst, city_map, classification_pie, sentiment_bar, histogram, boxplot, line_chart